In [1]:
import logging, scipy, dask, pandas, numpy, time, typing, asyncio
import dask.dataframe as ddf
import dask.bag as dbag
import dask.distributed as dask_distributed
from dask.distributed import Client
import distributed.diagnostics as daskdia
from sklearn.cluster import KMeans
from sklearn.externals import joblib

numpy.random.seed(201)
k = 100
representationThr = 3

/home/puhl/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# client = dask_distributed.Client()

# StreamZ Dask
client = await Client(processes=False)

# StreamZ Dask Async
# client = await Client(processes=False, asynchronous=True)

In [3]:
classes = [{'label': f'Class #{i}', 'mu': numpy.random.random() * 10, 'sigma': numpy.random.random() * 5} for i in range(5)]
def nextExample():
    cl = classes[numpy.random.randint(0, len(classes))]
    item = [float(numpy.random.normal(loc=cl['mu'], scale=cl['sigma'])) for i in range(40)]
    return {'label': cl['label'], 'item': item,}

def limit(stream, limit):
    msg = yield
    for target_list in range(limit):
        yield next(stream)

In [4]:
# %%timeit
# examples = dbag.from_sequence(limit(mkStream(), 10000))
# daskdia.progress(examples)

In [5]:
from dask.distributed import Client
from tornado.ioloop import IOLoop
import streamz
from streamz.dataframe import DataFrame as StreamzDataframe

exStream = streamz.Stream()
# exStream = streamz.Stream(asynchronous=True)
def aggOffile(accDF, example):
    if example is None:
        return accDF
    return accDF.append(pandas.DataFrame({'label': [example['label']], 'item': [example['item']]}))
offlineAgg = exStream.accumulate(aggOffile, start=pandas.DataFrame({'label': [], 'item': []}), returns_state=False)

# sdf = StreamzDataframe(source,)
# # buffer(5)
# dfStream = source.map(lambda ex: pandas.dataframe(data=ex))
# dfStream.accumulate().map(lambda ex: pandas.dataframe(data=ex)).scatter().map(increment).gather().sink()

ModuleNotFoundError: No module named 'streamz'

In [ ]:
print(offlineAgg)

In [ ]:
offlineAgg.sink(print)

In [ ]:
exStream.emit(nextExample())

In [ ]:
%%timeit
for i in range(10000):
    exStream.emit(nextExample())

In [ ]:
async def generate():
    for i in range(10000):
        await asyncio.sleep(10)
        await exStream.emit(nextExample())
    await exStream.emit(None)
# asyncio.get_loop().run(generate)

In [ ]:
class Cluster():
    def __init__(self, center=[]):
        self.center = center
        self.label = None
        self.n = 0
        self.lastExapleTMS = 0
        self.sumDistance = 0
        self.maxDistance = 0
    def dist(self, item):
        return scipy.spatial.distance.euclidean(self.center, item)
    def addExample(self, item, distance=None):
        self.n += 1
        if example.timestamp > self.lastExapleTMS:
              self.lastExapleTMS = example.timestamp
        if distance == None:
              distance = self.dist(item)
        self.sumDistance += distance
        if distance > self.maxDistance:
              self.maxDistance = distance

In [ ]:
def clustering(examples, label=None):
    representationThr = 3
    kmeans = KMeans( n_clusters = min(k, int(len(examples) / (3 * representationThr))) )
    with joblib.parallel_backend('dask'):
        kmeans.fit(examples)
    return [Cluster(center=centroid, label=label) for centroid in kmeans.cluster_centers_]
def closestCluster(item, clusters):
    return min([ (cl, cl.dist(item)) for cl in clusters ], key=lambda x: x[1])
def offline(label, group):
    representationThr = 3
    clusters = clustering(group, label)
    for ex in group:
        nearCl, dist = closestCluster(ex, clusters)
        nearCl.addExample(ex)
    return [cluster for cluster in clusters if cluster.n > representationThr]

In [ ]:
def group(i, downStream=streamz.Stream()):
    _, df = i
    return [ downStream.emit( (l, df[g['item']]) ) for l, g in df.groupby('label') ]
def mapOffline(i):
    label, group = i
    return offline(label, group)

trainingTrigger = streamz.Stream()
groups = streamz.Stream()

L = []
def nnnn(i):
    L.append(i)
trainingTrigger.zip(offlineAgg).sink(nnnn)
# grouped = trainingTrigger.zip(offlineAgg).map(group, downStream=groups)
# grouped.sink(lambda x: print('grouped {x!r} {t}'.format(x=x, t=type(x))))

# trained = groups.map(mapOffline)
# trained = groups.scatter().map(mapOffline).gather()
# trained.sink(lambda x: print('trained {x!r} {t}'.format(x=x, t=type(x))))

trainingTrigger.emit(True)
exStream.emit(nextExample())
trained.visualize(rankdir='LR')

In [ ]:
# trainingTrigger.emit(True)
# trained.sink(print)
L

In [ ]:
from streamz.dataframe import Random, DataFrame
%matplotlib inline

source = Random(freq='5ms', interval='100ms')
r = source.x.sum()
r.visualize()

In [ ]:
sdf = (source - 0.5).cumsum()
sdf.tail()

In [ ]:
p = (DataFrame({'raw': sdf.x,
                        'smooth': sdf.x.rolling('100ms').mean(),
                        'very-smooth': sdf.x.rolling('500ms').mean()})
     .plot(width=700)
    )

In [ ]:
import streamz
def increment(x):
    return x + 1

def decrement(x):
    return x - 1

s = streamz.Stream()
a = s.map(increment).sink(print)
b = s.map(decrement).sink(print)
b.visualize(rankdir='LR')